<h1>Calculating mean Average Precision (mAP)</h1>

In [2]:
import os
from mrcnn.model import MaskRCNN
# ROOT
ROOT = os.getcwd()
DATASET = os.path.join(ROOT,'dataset')

IMAGE_DIR =  os.path.join(DATASET,'images')
ANNOTATIONS_DIR = os.path.join(DATASET,'annotations')
LOG = os.path.join(ROOT,'logs')


/home/niko/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/niko/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/niko/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/niko/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWa

In [4]:

from cch_dataSet import CchDataset
train_set = CchDataset()

#  organize data
(images, annotations, path_ids) = train_set.prepare_dataset(IMAGE_DIR,ANNOTATIONS_DIR)
train_set.load_dataset(images,annotations,path_ids,True)
train_set.prepare()

# validation/test
test_set = CchDataset()
test_set.load_dataset(images,annotations,path_ids,False)
test_set.prepare()


In [5]:
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
from mrcnn.utils import compute_ap
from mrcnn.config import Config

from numpy import expand_dims, mean


class InferConfig(Config):
   # name of the configuration
    NAME = "cochonilha"
    # ch class + background class
    NUM_CLASSES = 3 + 1
    #otimizations
    BACKBONE = "resnet50"
    
    MAX_GT_INSTANCES = 100
    GPU_COUNT = 1
    # deixar igual em batch size
    IMAGES_PER_GPU = 1
    
    BATCH_SIZE = 1
    # amostras/batch
    # X2
    STEPS_PER_EPOCH = 112
    # validation steps
    VALIDATION_STEPS = 25
    
    DETECTION_MAX_INSTANCES = 100
    DETECTION_MIN_CONFIDENCE = 0.7
    DETECTION_NMS_THRESHOLD = 0.3
    
   
    IMAGE_CHANNEL_COUNT = 3
    IMAGE_MAX_DIM = 384
    
    IMAGE_META_SIZE = 16
    IMAGE_MIN_DIM = 300
    # IMAGE_MIN_SCALE = 0
    IMAGE_RESIZE_MODE = 'square'
    IMAGE_SHAPE = [300,300,3]

def evaluate_model(dataset, model, cfg):

  # cfg.BATCH_SIZE = 1
  # print(f'lengh = {cfg.BATCH_SIZE}')
 
  APs = []
  for image_id in dataset.image_ids:
        
		
    image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, cfg, image_id, use_mini_mask=False)

    
    scaled_image = mold_image(image, cfg)
    
    # sample = expand_dims(scaled_image, 0)
    
    yhat = model.detect([scaled_image], verbose=0)
		
    r = yhat[0]
		
    AP, precisions, recalls, overlaps = compute_ap(gt_bbox, gt_class_id, gt_mask, 
                                                          r["rois"], r["class_ids"], r["scores"], 
                                                          r['masks'], iou_threshold=0.5)
		
    APs.append(AP)

  mAP = mean(APs)
  return mAP

In [45]:
print(f'len-train: {train_set.num_images} len-test: {test_set.num_images}')

len-train: 138 len-test: 35


<h1> modelo #1 b-sm-rpn-anc</h2>

In [50]:
# os.sys.path.append('/home/niko/project/pibic_cch_maskRCNN_test/variant-b-sm-rpn-anc')
# from cch_config_variant_ import CchConfig

model_path = os.path.join(ROOT,'variant-b-sm-rpn-anc/model/cochonilha20230624T0150/mask_rcnn_cochonilha_0008.h5')

model = MaskRCNN(mode='inference', model_dir=LOG,config=InferConfig())
model.load_weights(model_path, by_name=True)

# train
train_mAP = evaluate_model(train_set,model,InferConfig)
# test
test_mAP = evaluate_model(test_set,model,InferConfig)

Re-starting from epoch 8


In [51]:
print(f'Precisão média treino: { train_mAP}')
print(f'Precisão média teste: { test_mAP}')

Precisão média treino: 0.06724372425312058
Precisão média teste: 0.05735007284830014


<h1> modelo #2 c-delta</h2>

In [8]:

model_path = os.path.join(ROOT,'variant-c-delta/model/cochonilha20230624T1314/mask_rcnn_cochonilha_0005.h5')

model = MaskRCNN(mode='inference', model_dir=LOG,config=InferConfig())
model.load_weights(model_path, by_name=True)

# train
train_mAP = evaluate_model(train_set,model,InferConfig)
# test
test_mAP = evaluate_model(test_set,model,InferConfig)

Re-starting from epoch 5


In [9]:
print(f'Precisão média treino: { train_mAP}')
print(f'Precisão média teste: { test_mAP}')

Precisão média treino: 0.037823336128032334
Precisão média teste: 0.030157139045851572


<h1> modelo #3 e-more-layers</h2>

In [ ]:

model_path = os.path.join(ROOT,'variant-e-more-layers/model/final/mask_rcnn_cochonilha_0004.h5')

model = MaskRCNN(mode='inference', model_dir=LOG,config=InferConfig())
model.load_weights(model_path, by_name=True)

# train
train_mAP = evaluate_model(train_set,model,InferConfig)
# test
test_mAP = evaluate_model(test_set,model,InferConfig)

In [7]:
print(f'Precisão média treino: { train_mAP}')
print(f'Precisão média teste: { test_mAP}')

Precisão média treino: 0.018449396514873682
Precisão média teste: 0.012996260518474238
